In [1]:
import numpy as np
import torch
from torch import nn
import torch.optim as optim
import matplotlib.pyplot as plt
from lstm_autoencoder_source import CNN_Autoencoder
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import os
import cv2
import pickle

In [6]:
kind = 'squat'
with open(f"{kind}_dataloader.pkl", "rb") as f:
    saved_data = pickle.load(f)

loaded_dataset = saved_data["dataset"]
dataloader = DataLoader(
    loaded_dataset,
    batch_size=saved_data["batch_size"],
    shuffle=saved_data["shuffle"],
)

In [7]:
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
model = CNN_Autoencoder().to(device)
def saveModel():
    torch.save(model.state_dict(), f'model/{kind}_train_l1.pt')
criterion = nn.L1Loss()
optimizer = torch.optim.RAdam(model.parameters(), lr=1e-2)
scheduler = CosineAnnealingLR(optimizer,T_max= 10, eta_min=1e-4)
# 학습 루프
epochs = 50
min_loss = 1e+10
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for data in dataloader:
        input = data.to(device)        
        optimizer.zero_grad()

        # 순전파
        output = model(input)

        # 손실 계산
        loss = criterion(output,input)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    mean_loss =running_loss / len(dataloader)
    print(f"Epoch [{epoch + 1}/{epochs}], Loss: {running_loss / len(dataloader)}")
    if min(mean_loss,min_loss) == mean_loss:
        min_loss = mean_loss
        print('model_saved')
        saveModel()
    

Epoch [1/50], Loss: 0.02285379934475191
model_saved
Epoch [2/50], Loss: 0.005327801156596162
model_saved
Epoch [3/50], Loss: 0.0053278008369077616
model_saved
Epoch [4/50], Loss: 0.005327800779348293
model_saved
Epoch [5/50], Loss: 0.005327800754716989
model_saved
Epoch [6/50], Loss: 0.005327800741753145
model_saved
Epoch [7/50], Loss: 0.005327800733197007
model_saved
Epoch [8/50], Loss: 0.0053278007287893
model_saved
Epoch [9/50], Loss: 0.005327800725159424
model_saved
Epoch [10/50], Loss: 0.005327800723344486
model_saved
Epoch [11/50], Loss: 0.005327800722825932
model_saved
Epoch [12/50], Loss: 0.005327800721270271
model_saved
Epoch [13/50], Loss: 0.005327800721270271
model_saved
Epoch [14/50], Loss: 0.005327800719196056
model_saved
Epoch [15/50], Loss: 0.005327800717121841
model_saved
Epoch [16/50], Loss: 0.005327800716603287
model_saved
Epoch [17/50], Loss: 0.005327800715566179
model_saved
Epoch [18/50], Loss: 0.005327800714529072
model_saved
Epoch [19/50], Loss: 0.0053278007140105